<a href="https://colab.research.google.com/github/gabbiurlaro/aml22-ego/blob/vae/augment_datset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INSTALL AND GIT


In [1]:
!rm -rf sample_data

!git clone https://github.com/gabbiurlaro/aml22-ego.git aml22-ego
!cd aml22-ego && git checkout vae

Cloning into 'aml22-ego'...
remote: Enumerating objects: 3595, done.
remote: Counting objects: 100% (240/240), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 3595 (delta 151), reused 194 (delta 118), pack-reused 3355
Receiving objects: 100% (3595/3595), 1.51 GiB | 33.70 MiB/s, done.
Resolving deltas: 100% (2624/2624), done.
Updating files: 100% (45/45), done.
Updating files: 100% (161/161), done.
Branch 'vae' set up to track remote branch 'vae' from 'origin'.
Switched to a new branch 'vae'


In [2]:
# Mount google drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Install conda

!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...


In [4]:
!mkdir -p /usr/local/envs/egovision
!tar xf /content/drive/MyDrive/egovision.tar.gz --directory=/usr/local/envs/egovision

In [17]:
%%bash
cd aml22-ego & git pull origin vae

Updating 0517763..392274a
Fast-forward
 utils/loaders.py | 6 ++++--
 1 file changed, 4 insertions(+), 2 deletions(-)


bash: line 1: cd: aml22-ego: No such file or directory
From https://github.com/gabbiurlaro/aml22-ego
 * branch            vae        -> FETCH_HEAD
   0517763..392274a  vae        -> origin/vae


# Augmentation

In [5]:
import pickle 
import numpy as np
import pandas as pd
import pywt
import torch
import os
import torch.nn.functional as F
from scipy.interpolate import CubicSpline
import random
import torchvision.transforms
from torch.utils.data import Dataset

train =  pd.DataFrame(pd.read_pickle('/content/aml22-ego/saved_features/ACTIONNET_EMG/EMG_no-clip_ActionNet_train.pkl'))
test =  pd.DataFrame(pd.read_pickle('/content/aml22-ego/saved_features/ACTIONNET_EMG/EMG_no-clip_ActionNet_test.pkl'))
original_train = pd.DataFrame(pd.read_pickle('/content/drive/MyDrive/train_val_EMG/ActionNet_train.pkl'))
original_test = pd.DataFrame(pd.read_pickle('/content/drive/MyDrive/train_val_EMG/ActionNet_test.pkl'))
original_train

,uid,subject,verb,verb_class,description,start_frame,stop_frame,start_ts,end_ts,myo_left_readings,myo_right_readings
0,0,S08_1,Spread,0,Spread jelly on a bread slice,81240,81898,00:45:07.987279,00:45:29.929329,"[[4, 12, 21, -21, 3, -2, -5, 7], [-6, -5, -5, ...","[[-1, 9, 36, 34, 6, 3, 5, -2], [3, 12, 26, -9,..."
1,1,S04_1,Get/Put,1,Get/replace items from refrigerator/cabinets/d...,42403,44509,00:23:33.433146,00:24:43.612856,"[[2, 2, -11, -8, -2, -7, 19, 5], [24, 17, -7, ...","[[-9, 11, 17, -38, -13, -6, 2, -5], [11, -4, 2..."
2,2,S03_1,Clear,2,Clear cutting board,15308,15741,00:08:30.234442,00:08:44.687263,"[[-9, -14, -2, -6, 3, 3, 2, 19], [11, 8, -12, ...","[[-26, -22, 17, 5, -23, -22, -20, -24], [19, 2..."
3,3,S08_1,Clear,2,Clear cutting board,64690,64849,00:35:56.322172,00:36:01.618174,"[[-7, -10, -6, 9, 7, -6, -18, 5], [17, -3, -18...","[[13, 19, -52, -15, -13, -3, -2, -2], [30, 16,..."
4,4,S04_1,Slice,3,Slice a cucumber,35308,36764,00:19:36.920691,00:20:25.454613,"[[18, 20, -8, 8, -16, 7, 9, 12], [5, -21, -35,...","[[7, 13, 45, 14, -4, 1, 2, 4], [0, 2, -14, -10..."
...,...,...,...,...,...,...,...,...,...,...,...
521,521,S05_2,Clean,4,Clean a pan with a sponge,70522,70836,00:39:10.708422,00:39:21.176074,"[[4, -10, -1, 8, 3, -8, -8, 5], [18, -3, -4, -...","[[-2, -15, -46, 16, 6, -1, -2, -1], [4, 5, -10..."
522,522,S09_2,Clean,4,Clean a plate with a sponge,80762,81079,00:44:52.036009,00:45:02.629240,"[[-24, -10, -2, 1, 0, -9, -2, -25], [-20, -10,...","[[-7, -1, 8, 3, 10, 18, 14, -5], [4, 6, 27, 11..."
523,523,S03_2,Spread,0,Spread almond butter on a bread slice,6954,7794,00:03:51.795153,00:04:19.795153,"[[5, 4, 3, 0, 6, 9, 12, 7], [-4, -3, -1, -2, 2...","[[2, 12, 10, 6, -6, -2, -1, -1], [24, 0, 14, -..."
524,524,S02_2,Slice,3,Slice a cucumber,36379,37997,00:20:12.603901,00:21:06.561359,"[[-13, 3, 3, -4, 4, 19, 23, 4], [-3, 5, 19, -9...","[[-63, -94, -12, -19, -17, 2, 8, -7], [18, 100..."


In [7]:
def wavelet_decomposition(signal, wavelet_name, decomposition_level, detail_factor):
    coeffs = pywt.wavedec(signal, wavelet=wavelet_name, level=decomposition_level)
    cA, cD = coeffs[0], coeffs[1:]  # Approximation and detail coefficients
    
    # Modify detail coefficients
    cD_modified = [detail_factor * cd for cd in cD]
    
    # Reconstruct the augmented signal
    augmented_coeffs = [cA] + cD_modified
    augmented_signal = np.array(pywt.waverec(augmented_coeffs, wavelet=wavelet_name))
    
    return augmented_signal


class WaveletDecompositionTransform:
    def __init__(self, wavelet_name, decomposition_level, detail_factor, num_clips):
        self.wavelet_name = wavelet_name
        self.decomposition_level = decomposition_level
        self.detail_factor = detail_factor
        self.num_clips = num_clips
    
    def __call__(self, sample):
        signals = sample['features_EMG']
        augmented_signals = []
        for i in range(self.num_clips):
          augmented_signals.append(wavelet_decomposition(torch.Tensor(signals[i]), self.wavelet_name, self.decomposition_level, self.detail_factor))
        # Create a new augmented sample
        augmented_sample = {
            'features_EMG': np.array(augmented_signals),
            'label': sample['label'],
            'uid': sample['uid'],
            'untrimmed_video_name': sample['untrimmed_video_name']
        }
        
        return augmented_sample

In [8]:
def magnitude_warping(signal, variance=0.01):
    T = signal.size(0)
    t = torch.linspace(0, 1, T)  # Equidistant time points
    r = torch.randn(T)  # Random numbers from a normal distribution
    r = torch.clamp(r, -2.0, 2.0)  # Limit the range of random numbers to avoid extreme warping
    
    # Generate a smooth curve using cubic splines
    spline = CubicSpline(t, r)
    cubic_spline = torch.from_numpy(spline(t)).float()
    
    # Elementwise product of the interpolated curve with the signal
    warped_signal = np.array(signal * (1.0 + variance * cubic_spline))

    return warped_signal

class MagnitudeWarpingTransform:
    def __init__(self, variance, num_clips):
        self.variance= variance
        self.num_clips = num_clips
    
    def __call__(self, sample):
        signals = sample['features_EMG']
        augmented_signals = []
        for i in range(self.num_clips):
          augmented_signals.append(magnitude_warping(torch.Tensor(signals[i]), variance=self.variance))
        # Create a new augmented sample
        augmented_sample = {
            'features_EMG': np.array(augmented_signals),
            'label': sample['label'],
            'uid': sample['uid'],
            'untrimmed_video_name': sample['untrimmed_video_name']
        }
        
        return augmented_sample

# Example usage
#signal = torch.randn(1024)  # Assuming input signal of size 1024
#warped_signal = magnitude_warping(signal, variance=0.01)

In [9]:
class SlidingWindowTransform:
    def __init__(self, window_length, overlap=False, num_clips=5):
        self.window_length = window_length
        self.overlap = overlap
        self.num_clips = num_clips

    def __call__(self, x):
        signals = x['features_EMG']
        for i in range(self.num_clips):
          num_segments = len(signals) // self.window_length
          print( len(signals))
          if self.overlap:
              stride = self.window_length // 2
          else:
              stride = self.window_length
          augmented_signals = []
          for i in range(num_segments):
              start = i * stride
              end = start + self.window_length
              segment = signals[start:end]
              augmented_signals.append(segment)
        augmented_sample = {
              'features_EMG': torch.stack(augmented_signals),
              'label': x['label'],
              'uid': x['uid'],
              'untrimmed_video_name': x['untrimmed_video_name']
          }
        
        return augmented_sample


In [22]:
%%bash
cd ../aml22-ego
ls

action-net
augment_datset.ipynb
colab_runner.ipynb
configs
Experiment_logs
generate_video_from_frame.ipynb
img
img_VAE.png
jj.mdx
models
My__colab_runner.ipynb
pretrained_i3d
README.md
requirements.yaml
results_classifier
RGB_SEMG.py
saved_features
saved_models
save_feat_actionnet.py
save_feat_epic.py
scripts
Second_colab_runner.ipynb
tasks
train_AE_features_clip.py
train_classifier_EMG.py
train_classifier_LSTM.py
train_classifier_MLP.py
train_classifier_multimodal.py
train_classifier_TRN.py
train_VAE_EMG.py
train_VAE_features_clip.py
train_VAE_features_EMG.py
train_val
transform_annotations.ipynb
utils
visualize_feature.ipynb


In [31]:
from utils.loaders import ActionNetDataset
sigma = 0.1
wavelet_name = 'db7' #Wavelet name (e.g., Daubechies 4)
decomposition_level = 5 # # Number of decomposition levels
detail_factor = 0 # Scaling factor for modifying detail coefficients
  
num_clips = 5
batch_Size = 1

train =  ActionNetDataset('ActionNet', ['EMG'], 'train', {'stride': 2, 'annotations_path':'/content/drive/MyDrive/train_val_EMG'}, {'EMG': 32}, 5, {'EMG': False},
                                                                       None, load_feat=True)
#pd.DataFrame(pd.read_pickle('/content/aml22-ego/saved_features/ACTIONNET_EMG/EMG_no-clip_ActionNet_train.pkl'))
test =  ActionNetDataset('ActionNet', ['EMG'], 'test', {'stride': 2, 'annotations_path':'/content/drive/MyDrive/train_val_EMG'}, {'EMG': 32}, 5, {'EMG': False},
                                                                       None, load_feat=True)
train.list_file


AttributeError: ignored

In [ ]:

wavelet_transform = WaveletDecompositionTransform(wavelet_name, decomposition_level, detail_factor, num_clips)
magnitude_warp_transform = MagnitudeWarpingTransform(sigma, num_clips)
#sw_transform = SlidingWindowTransform(window_length=100, overlap=True, num_clips=num_clips)

transforms_A = [
    torchvision.transforms.Compose([wavelet_transform, magnitude_warp_transform]),
     torchvision.transforms.Compose([magnitude_warp_transform ]), 
     torchvision.transforms.Compose([wavelet_transform]),
     torchvision.transforms.Compose([magnitude_warp_transform, wavelet_transform])
    ] 
out_train = {}
out_test = {}
outs = [out_train, out_test]
datasets = [train, test]

for data_id in range(len(datasets)):
  for aug_set in range(4):
    augmented_samples = [transforms_A[aug_set](datasets[data_id]['features'][i]) for i in range(len(datasets[data_id]['features']))]
    outs[data_id][aug_set] = {'features': list(augmented_samples)}
    print(len(outs[data_id][aug_set]['features']))
  #for feats in datasets[data_id]['features']:

ts = ['WD-MW', 'MW', 'WD', 'MW-WD']
for i in range(4):
  filename = './saved_features/aug/ActionNet_augmented_clips_' + ts[i]
  with open(f"{filename}_train.pkl", "wb") as file:
            pickle.dump(outs[0][i], file)
  with open(f"{filename}_test.pkl", "wb") as file:
             pickle.dump(outs[1][i], file)

In [ ]:
from google.colab import files
!zip -r /content/feats_def.zip /content/aml22-ego/saved_features/aug
files.download('/content/feats_def.zip')

# Plot features

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
num_clips = 5
# EPIC-KITCHEN and ActionNet

labels = {'EK':{
        0 : "take (get)",
        1 : "put-down (put/place)",
        2 : "open",
        3 : "close",
        4 : "wash (clean)",
        5 : "cut",
        6 : "stir (mix)",
        7 : "pour"
}, 'AN': {
        0 : "Spread",
        1 : "Get/Put",
        2 : "Clear",
        3 : "Slice",
        4 : "Clean",
        5 : "Pour",
        6 : "Load",
        7 : "Peel",
        8 : "Open/Close",
        9 : "Set",
        10 : "Stack",
        11 : "Unload"

}}

colors = {'EK': {
        0 : "#A52A2A",
        1 : "#DAA520",
        2 : "#FF7F50",
        3 : "#7BC8F6",
        4 : "#FFFF14",
        5 : "#76FF7B",
        6 : "#13EAC9",
        7 : "olive"
}, 'AN': {
        0 : "#A52A2A",
        1 : "#DAA520",
        2 : "#FF7F50",
        3 : "#7BC8F6",
        4 : "#FFFF14",
        5 : "#76FF7B",
        6 : "#13EAC9",
        7 : "olive",
        8 : "palegreen",
        9 : "lightpink",
        10 : "darkmagenta",
        11 : "cadetblue"
}}

In [ ]:
data_original = pd.DataFrame(pd.read_pickle('/content/aml22-ego/saved_features/ACTIONNET_EMG/EMG_no-clip_ActionNet_train.pkl')["features"])
data = pd.DataFrame(pd.read_pickle("/content/aml22-ego/saved_features/reconstructed/AUG_VAE_0.001_2023-05-24 16:15:44.696068_train.pkl")["features"])


In [ ]:
data.iloc[0]
ts = ['WD-MW', 'MW', 'WD', 'MW-WD']
data_augmented_train = []
for i in range(4):
  data_augmented_train.append(pd.read_pickle('/content/aml22-ego/saved_features/aug/ActionNet_augmented_clips_'+ ts[i]+'_train.pkl'))
data = pd.DataFrame(data_augmented_train[3]['features'] +data_augmented_train[2]['features']+
                   data_augmented_train[1]['features'] +
                    data_augmented_train[0]['features'] + data_original['features'])
len(data)

In [ ]:
# plot emg features
print(len(data.iloc[0]['features_EMG']))
features = np.array([data.iloc[i].features_EMG[num_clips//2] for i in range(len(data))])
reduced = TSNE().fit_transform(features)
data['x'] = reduced[:, 0]
data['y'] = reduced[:, 1]
for i in range(12): # ek has 8 classes
    filtered = data[data["label"] == i]
    # compute the central frame
    plt.scatter(filtered['x'], filtered['y'], c=colors['AN'][i], label=labels['AN'][i])
plt.legend()
plt.title('EMG train features')
plt.show()

# Train VAE

In [ ]:
%%bash
#VAE EMG train and save

cd aml22-ego && git pull origin vae

PYTHON_PATH=/usr/local/envs/egovision/bin/python

$PYTHON_PATH train_VAE_features_EMG.py action="train_and_save" \
  name="VAE_EMG_2 full-aug lr1e-3 wkld1 sum" \
  config=configs/VAE_save_feat_EMG.yaml \
  dataset.shift=ActionNet-ActionNet \
  wandb_name='vae' \
  wandb_dir='Experiment_logs'  \
  dataset.RGB.data_path=../ek_data/frames \
  dataset.EMG.features_name='ACTIONNET_EMG/EMG_no-clip' \
  models.EMG.model='VAE' \
  models.EMG.lr=1e-3